In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
import pandas as pd
from pandas.plotting import autocorrelation_plot
from pandas import DataFrame



import seaborn as sns
import statsmodels.api as sm 
from statsmodels.tsa.stattools import adfuller 
from statsmodels.stats.diagnostic import acorr_ljungbox 
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
from statsmodels.tsa.arima_model import ARIMA 
from statsmodels.tsa.arima_model import ARMA 
from statsmodels.stats.stattools import durbin_watson 
from statsmodels.graphics.api import qqplot 

from sklearn.metrics import mean_squared_error

from statsmodels.tsa.statespace.sarimax import SARIMAX

from math import sqrt



In [2]:

# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)


In [3]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))

In [ ]:
data = pd.read_csv("/Users/rich/PycharmProjects/GDP_forecast/WEO_Data.csv")
data=data.drop(['Units', 'Scale','Estimates Start After','Unnamed: 51'],axis=1)
data=data.drop([5,6],axis=0)
data.set_index('Country', inplace=True)
data=data.T
year=pd.period_range('1980','2026',freq='Y')
data.index= year
df=data['1980':'2020']
Estimates_data=data['2021':'2026']
print(df)
print(Estimates_data)
import warnings
warnings.filterwarnings("ignore")
for i in df.columns:
    X=df[i]
    print(i)
    evaluate_models(X.values, p_values, d_values, q_values)

Country   China  Germany   India  Japan  United States
1980      7.910    1.272   5.281  3.181         -0.257
1981      5.100    0.110   6.006  4.209          2.538
1982      9.000   -0.788   3.476  3.312         -1.802
1983     10.800    1.555   7.289  3.523          4.584
1984     15.200    2.826   3.821  4.502          7.237
1985     13.501    2.192   5.254  5.233          4.169
1986      8.900    2.417   4.777  3.327          3.463
1987     11.700    1.469   3.965  4.731          3.460
1988     11.200    3.736   9.628  6.785          4.177
1989      4.200    3.913   5.947  4.858          3.673
1990      3.888    5.723   5.534  4.893          1.886
1991      8.401    5.011   1.057  3.417         -0.108
1992     14.234    1.920   5.482  0.848          3.522
1993     14.023   -0.977   4.750 -0.518          2.753
1994     13.031    2.399   6.659  0.876          4.029
1995     11.012    1.541   7.575  2.631          2.684
1996      9.982    0.816   7.550  3.134          3.772
1997      